# Business Analytics und Künstliche Intelligenz

Prof. Dr. Jürgen Bock & Maximilian-Peter Radtke

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Ein erstes Modell

In dieser Einheit soll es darum gehen ein erstes Modell zu erstellen um eine Regression durchzuführen. Hierfür werden wir das Python Package `statsmodels` verweden. Eine lineare Regression kann auch über `scikit-learn` angewandt werden, aber hier werden nicht alle statistischen Kennzahlen angegeben, welche sehr wichtig für die Interpretation der Ergebnisse sind.

### Statsmodels


[Statsmodels]("https://www.statsmodels.org/stable/index.html") ist ein Pythonmodul, welches Klassen und Funktionen für die Schätzung verschiedener statistsicher Modelle und Tests bereitstellt.

Falls Sie das erste Mal mit statsmodels arbeiten, müssen Sie das statsmodels Paket zunächst [installieren]("https://www.statsmodels.org/stable/install.html"). Dies können Sie mit dem Befehl `conda install statsmodels` über die Konsole machen (der Ort von dem Sie auch Jupyter Notebook starten). Alternativ können Sie auch direkt über Jupyter das neue Paket installieren. Dazu müssen Sie nur die folgende Zelle ausführen.

In [ ]:
# Installation eines Conda Pakets im aktuellen Kernel
import sys
!conda install --yes --prefix {sys.prefix} statsmodels

Nun können Sie das Paket importieren.

In [ ]:
import statsmodels.api as sm

Zunächst wollen wir uns denn Datensatz zu den Werbeausgaben aus der Vorlesung genauer ansehen. 

## Datenverständnis

In [ ]:
ad = pd.read_csv('Advertising.csv', index_col=0)

In [ ]:
ad.head()

In [ ]:
pd.plotting.scatter_matrix(ad, figsize=(15,15))
plt.show()

Wie bereits in der Vorlesung besprochen, sehen wir einen starken linearen Zusammenhang zwischen TV-Werbesausgaben und Sales. Diesen Zusammenhang werden wir nutzen, um ein erstes lineares Modell zu erzeugen.

Um Statsmodels hierfür zu nutzen, müssen wir unseren Daten zunächst eine Spalte mit dem Wert 1 hinzufügen, welche als $\beta_0$ genutzt wird.

## Datenvorbereitung

In [ ]:
ad['intercept'] = 1

In [ ]:
ad.head()

Als nächstes initialisieren wir das Modell für die lineare Regression. Hierfür nutzen wir `OLS`, was für **O**rdinary **L**east **S**quares - Methode der kleinsten Quadrate steht. Dafür übergeben wir der Methode die Zielvariable und den Prädiktor TV, inklusive unserem Dummy Wert für $\beta_0$. 

## Modellierung

In [ ]:
modTV = sm.OLS(ad.sales, ad.loc[:,['TV', 'intercept']])

In [ ]:
modTV

Um die Parameter $\hat{\beta}_0$ und $\hat{\beta}_1$ zu schätzen rufen wir von unserem initialisierten Modell die Methode `fit` auf.

In [ ]:
resTV = modTV.fit()

Die Parameter sind nun geschätzt und wir können uns die Zusammenfassung der Resultate anschauen.

In [ ]:
resTV.summary()

Als nächstes können wir unser Modell nutzen um Vorhersagen zu treffen. Hierfür nutzen wir die Methode `predict` und Plotten sie zusammen mit den bekannten Daten.

In [ ]:
plt.scatter(ad.TV, ad.sales, s=10)
plt.plot(ad.TV, resTV.predict(ad.loc[:,['TV', 'intercept']]), c='red')
plt.xlabel('TV')
plt.ylabel('Sales')
plt.show()

Jetzt schauen wir uns das Modell mit allen Variablen an.

In [ ]:
modAll = sm.OLS(ad.sales, ad.loc[:,['TV', 'radio', 'newspaper', 'intercept']])
resAll = modAll.fit()
resAll.summary()

Wenn wir Interaktionen zwischen Variablen modellieren möchten, müssen wir diese in unserem Ausgangsdatensatz hinterlegen. Hierfür erzeugen wir neue Spalten, welche der Interaktion entsprechen. Der multiplikative Zusammenhang zwischen TV und radio wird wie folgt dargestellt.

In [ ]:
ad['TVXradio'] = ad.TV * ad.radio

Diese neue Spalte können wir direkt in ein neues Modell aufnehmen.

In [ ]:
modInter = sm.OLS(ad.sales, ad.loc[:,['TV', 'radio', 'TVXradio', 'intercept']])
resInter = modInter.fit()
resInter.summary()

## Bias Variance Tradeoff

Mit diesem Datensatz können wir uns auch nochmals den Bias Variance Tradeoff ansehen. Dazu starten wir zunächst mit dem simplen Modell, mit dem der Umsatz nur durch die TV-Ausgaben vorhergesagt wird. Danach machen wir das Modell stetig flexibler (erhöhen sozusagen die Varianz) indem wir das Polynom um eine Ordnung erhöhen.

Entsprechend haben wir die Modelle:

TV_1: $y = \beta_0 + \beta_1 x$ \
TV_2: $y = \beta_0 + \beta_1 x + \beta_2 x^2$ \
TV_3: $y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3$ \
...

Wir evaluieren wie gut die verschiedenen Modelle sind, indem wir $R^2$ auf Daten berechnen, die vorher nicht für das Training genutzt wurden.

In [ ]:
import numpy as np
def r_squared(y_true, y_pred):
    # Calculate the mean of the actual values
    y_mean = np.mean(y_true)
    # Calculate the total sum of squares (TSS)
    tss = np.sum((y_true - y_mean) ** 2)
    # Calculate the residual sum of squares (RSS)
    rss = np.sum((y_true - y_pred) ** 2)
    # Calculate R-squared
    return 1 - (rss / tss)   

# Loop through specific fraction of data
for n_frac in [0.9, 0.5, 0.1]:
    # Only use part of the data for training
    sampled = ad.sample(frac=n_frac, random_state=12)
    # Use the rest for testing
    not_sampled = ad.drop(sampled.index)
    
    # Initialize plot
    fig, ax = plt.subplots(figsize=(10,5))
    ax.scatter(not_sampled.TV, not_sampled.sales, color='grey', alpha=0.5, label='Train data')
    ax.scatter(sampled.TV, sampled.sales, color='blue', alpha=0.5, label='Test data')
    print(f'{int(n_frac*ad.shape[0])} of {ad.shape[0]} datapoints used for training')

    # Initialize model columns with "intercept" column
    model_columns = ['intercept']

    # Loop through polynomials of degree 1 to 9
    for i in range(1, 10):
        # Name of model
        col_name = f'TV_{i}'
        model_columns.append(col_name)
        # Add ith polynomial
        sampled[col_name] = sampled.TV ** i
        not_sampled[col_name] = not_sampled.TV ** i
        # Sort dataframe for plotting
        sampled = sampled.sort_values(by='TV', ascending=True)
        not_sampled = not_sampled.sort_values(by='TV', ascending=True)
        # Create model
        tmp_model = sm.OLS(sampled.sales, sampled.loc[:, model_columns])
        tmp_res = tmp_model.fit()
        # Output results
        # Predictions
        y_pred_test = tmp_res.predict(not_sampled.loc[:, model_columns])
        y_true_test = not_sampled.sales
        y_pred_train = tmp_res.predict(sampled.loc[:, model_columns])
        y_true_train = sampled.sales
        ax.plot(not_sampled.TV, y_pred_test, label=i)
        ax.set_xlabel('TV')
        ax.set_ylabel('Sales')
        r2_train = r_squared(y_true_train, y_pred_train)
        r2_test = r_squared(y_true_test, y_pred_test)
        print(f"{col_name} R2 Train: {r2_train:.3f} | R2 Test: {r2_test:.3f}")
    ax.legend()
    plt.show()
    
    print('------------')

## Auto Datensatz

Weiter machen wir mit dem Auto Datensatz von letzter Übung. Wir fügen auch hier direkt eine Spalte für $\beta_0$ hinzu.

In [ ]:
auto = pd.read_csv('Auto_clean.csv')
auto['intercept'] = 1
auto.head()

Bevor wir mit diesem Datensatz weiter arbeiten können, müssen wir Dummyvariablen für die qualitative Variable origin erstellen. Pandas stellt hierfür die Funktion `get_dummies` bereit, welche uns die händische Arbeit abnimmt. Mittels dem Parameter `drop_first` stellen wir ein, dass eine Variable weniger als die Anzahl der Ausprägungen hinzugefügt werden.

In [ ]:
dummyVar = pd.get_dummies(auto.origin, prefix='origin', drop_first=True, dtype=int)
dummyVar

Mithilfe von `concat` können wir die neuen Spalten dem alten Datensatz hinzufügen. Um sicherzugehen, dass die neuen Werte auch wirklich als Spalten (und nicht als Zeilen) hinzugefügt werden, übergeben wir den Parameter `axis=1`.

In [ ]:
autoDum = pd.concat([auto, dummyVar], axis=1)
autoDum.head()

Nun bauen wir ein Modell nur mit den Dummy Variablen mit dem Zielwert mpg.

In [ ]:
modAutoOrig = sm.OLS(autoDum.mpg, autoDum.loc[:,["origin_2", "origin_3", "intercept"]])
resAutoOrig = modAutoOrig.fit()
resAutoOrig.summary()

---

# Übungsaufgaben

## Aufgabe 1

Erstellen Sie eine einfach Regression mit Prädiktor **horsepower** und Zielwert **mpg** auf Basis des Auto-Datensatzes. Geben Sie sich die Zusammenfassung des Modells aus.
* Was fällt Ihnen auf? Zum Beispiel:
    * Gibt es einen Zusammenhang zwischen Prädiktor und Zielvariable?
    * Wie stark ist der Zusamenhang zwischen Prädiktor und Zielvariable?
    * Ist der Zusammenhang positiv oder negativ?
    * Was ist der vorhergesagte Wert von mpg für horsepower = 95?
* Plotten Sie die Regressionslinie und die zugrundeliegenden Daten in einem Plot.

In [ ]:
auto = pd.read_csv('Auto_clean.csv')
auto['intercept'] = 1

In [ ]:
autoMod = sm.OLS(auto.mpg, auto[['horsepower', 'intercept']])
autoRes = autoMod.fit()
autoRes.summary()

In [ ]:
autoRes.predict([95,1])

In [ ]:
plt.scatter(auto.horsepower, auto.mpg, s=10)
plt.plot(auto.horsepower, autoRes.predict(auto[['horsepower', 'intercept']]), color='red')
plt.xlabel('Horsepower')
plt.ylabel('mpg')

## Aufgabe 2

Benutzen Sie wieder den Auto-Datensatz und **mpg** als Zielvariable.
* Erstellen Sie eine lineare Regression und nutzen Sie alle Variablen.
    * Gibt es einen Zusammenhang zwischen den Prädiktoren und der Zielvariable?
    * Welche Prädiktoren scheinen einen signifikanten Einfluss zu haben?
    * Was suggeriert der Koeffizient für Jahr?
* Schätzen Sie eine Regression mit Interaktionen (Multiplikation oder Division). Sind irgendwelche Interaktionen statistisch signifikant?
* Nutzen Sie andere Transformationen für die Variabeln in der Regression, z.B. $X^2, \log{(X)}, \sqrt{X}$. Beschreiben Sie ihre Erkenntnisse.

In [ ]:
dummyVar = pd.get_dummies(auto.origin, prefix='origin', drop_first=True, dtype=int)
autoDum = pd.concat([auto, dummyVar], axis=1)

In [ ]:
autoDum.columns

In [ ]:
autoAllMod = sm.OLS(autoDum.mpg, autoDum.drop(['mpg', 'name', 'origin'], axis=1))
autoAllRes = autoAllMod.fit()
autoAllRes.summary()

In [ ]:
autoDum['CylinderXDisplacement'] = autoDum.cylinders * autoDum.displacement
autoDum['DisplacementXWeight'] = autoDum.displacement * autoDum.weight
autoInterMod = sm.OLS(
    autoDum.mpg,
    autoDum[['CylinderXDisplacement', 'DisplacementXWeight', 'cylinders', 'weight', 'displacement', 'intercept']]
)
autoInterRes = autoInterMod.fit()
autoInterRes.summary()

In [ ]:
autoDum['horsepower2'] = auto.horsepower * auto.horsepower
autoHP2Mod = sm.OLS(autoDum.mpg, autoDum[['horsepower', 'horsepower2', 'intercept']])
autoHP2Res = autoHP2Mod.fit()
autoHP2Res.summary()

In [ ]:
plt.scatter(autoDum.horsepower, autoDum.mpg, s=10)
plt.plot(
    autoDum.sort_values(by='horsepower').horsepower,
    autoHP2Res.predict(autoDum[['horsepower', 'horsepower2', 'intercept']].sort_values(by='horsepower')),
    color='red'
)
plt.xlabel('Horsepower')
plt.ylabel('mpg')
plt.show()